按照到期月分做分類，最近到期(本月或下個月)的歸類為NearbyMonth(近月)，以此類推，特別須知的是，因為每月的到期日是有規定在哪一天，所以近月到期的未必就是在31天內，也有可能超過30天。

In [1]:
import numpy as np
import pandas as pd
import os
def slash_tran(path):#檔案路徑字串可以允許不同的斜線(\ 或 / 都可以)，此程式會將他們統一轉成反斜線('\\')輸出
    dirs = path.split('/')
    path_new = dirs[0]
    for dir in dirs[1:]:
        path_new = path_new + '\\' + dir
    return path_new
def path_tran(old_path, top_path, new_dir):
    #top_path = './../dir1/dir2/'
    #old_path = './../dir1/dir2/dir3\\dir4\\\\file'
    #new_dir = 'New'
    #輸出為 '.\..\dir1\dir2\New\dir4\file'
    path = slash_tran(old_path)
    top_path = slash_tran(top_path)
    top_path_list = np.array(top_path.split('\\'))
    top_path_list = list(top_path_list[np.not_equal(top_path_list, '')])

    old_dir_idx = len(top_path_list)
    new_path_list = np.array(path.split('\\'))
    new_path_list = list(new_path_list[np.not_equal(new_path_list, '')])

    new_path_list[old_dir_idx] = new_dir
    new_path = new_path_list[0]
    for dir in new_path_list:
        new_path = new_path + '\\' + dir
    return new_path


top_path = './../..\\'
Data_path = top_path + '\Data\\'

AllExpiry_path = Data_path + '\AllExpiry'
file_name = 'IV.csv'
IV_data = pd.read_csv(AllExpiry_path + '\\' +  file_name, encoding='Big5', index_col=False)
expiry_column_name = '到期月份種類'
expirys = np.unique(np.array(IV_data[expiry_column_name]))




for expiry in expirys:
    if expiry not in os.listdir(Data_path):
        os.mkdir('{}\{}'.format(Data_path, expiry))

    for path, dirs, files in os.walk(AllExpiry_path):
        new_path = path_tran(path, Data_path, expiry)
        for dir in dirs:
            if dir not in os.listdir(new_path):
                os.mkdir(new_path + '\\' + dir)
        for file in files:
            IV_data = pd.read_csv(path + '\\' + file, encoding='Big5', index_col=False)
            IV_expiry_data = IV_data[IV_data[expiry_column_name] == expiry]
            IV_expiry_data.to_csv(new_path  + '\\' + file, encoding='Big5', index=False)

以到期天數做篩選，篩出指定到期日以內的資料。我希望先篩選出近月30天內的資料，但值得注意的是，次近月到期的資料，其實也有可能是在30天內的(有可能近月到期2天，次近月到期30天)，所以必先篩出分類為NearbyMonth的資料，再篩出30天以內的資料

In [2]:
import numpy as np
import pandas as pd
import os
def slash_tran(path):#檔案路徑字串可以允許不同的斜線(\ 或 / 都可以)，此程式會將他們統一轉成反斜線('\\')輸出
    dirs = path.split('/')
    path_new = dirs[0]
    for dir in dirs[1:]:
        path_new = path_new + '\\' + dir
    return path_new
def path_tran(old_path, top_path, new_dir):
    #top_path = './../dir1/dir2/'
    #old_path = './../dir1/dir2/dir3\\dir4\\\\file'
    #new_dir = 'New'
    #輸出為 '.\..\dir1\dir2\New\dir4\file'
    path = slash_tran(old_path)
    top_path = slash_tran(top_path)
    top_path_list = np.array(top_path.split('\\'))
    top_path_list = list(top_path_list[np.not_equal(top_path_list, '')])

    old_dir_idx = len(top_path_list)
    new_path_list = np.array(path.split('\\'))
    new_path_list = list(new_path_list[np.not_equal(new_path_list, '')])

    new_path_list[old_dir_idx] = new_dir
    new_path = new_path_list[0]
    for dir in new_path_list:
        new_path = new_path + '\\' + dir
    return new_path


top_path = './../..\\'
Data_path = top_path + '\Data\\'

AllExpiry_path = Data_path + '\AllExpiry'
file_name = 'IV.csv'
IV_data = pd.read_csv(AllExpiry_path + '\\' +  file_name, encoding='Big5', index_col=False)

expiry_type_column_name = '到期月份種類'
expiry_types = ['NearbyMonth']

expiry_days_column_name = '到期天數'
expiry_days = [30]




for i in range(len(expiry_days)):
    if 'in{}days'.format(expiry_days[i]) not in os.listdir(Data_path):
        os.mkdir('{}\{}'.format(Data_path, 'in{}days'.format(expiry_days[i])))

    for path, dirs, files in os.walk(AllExpiry_path):
        new_path = path_tran(path, Data_path, 'in{}days'.format(expiry_days[i]))
        for dir in dirs:
            if dir not in os.listdir(new_path):
                os.mkdir(new_path + '\\' + dir)
        for file in files:
            IV_data = pd.read_csv(path + '\\' + file, encoding='Big5', index_col=False)
            IV_expiry_data = IV_data[IV_data[expiry_type_column_name] == expiry_types[i]]
            IV_expiry_data = IV_expiry_data[IV_data[expiry_days_column_name] <= expiry_days[i]]
            IV_expiry_data.to_csv(new_path  + '\\' + file, encoding='Big5', index=False)

C:\Users\bgpa4\AppData\Local\Temp\ipykernel_24836\1728939491.py:59: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  IV_expiry_data = IV_expiry_data[IV_data[expiry_days_column_name] <= expiry_days[i]]
C:\Users\bgpa4\AppData\Local\Temp\ipykernel_24836\1728939491.py:59: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  IV_expiry_data = IV_expiry_data[IV_data[expiry_days_column_name] <= expiry_days[i]]
C:\Users\bgpa4\AppData\Local\Temp\ipykernel_24836\1728939491.py:59: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  IV_expiry_data = IV_expiry_data[IV_data[expiry_days_column_name] <= expiry_days[i]]
C:\Users\bgpa4\AppData\Local\Temp\ipykernel_24836\1728939491.py:59: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  IV_expiry_data = IV_expiry_data[IV_data[expiry_days_column_name] <= expiry_days[i]]
